# Doodles

Exploring the dataset.

In [ ]:
import pandas as pd

df = pd.read_csv("IW_os_placenames.csv")
df.head()

,ID,NAMES_URI,NAME1,NAME1_LANG,NAME2,NAME2_LANG,TYPE,LOCAL_TYPE,GEOMETRY_X,GEOMETRY_Y,...,COUNTY_UNITARY,COUNTY_UNITARY_URI,COUNTY_UNITARY_TYPE,REGION,REGION_URI,COUNTRY,COUNTRY_URI,RELATED_SPATIAL_OBJECT,SAME_AS_DBPEDIA,SAME_AS_GEONAMES
0,SU22.csv:osgb4000000074770417,http://data.ordnancesurvey.co.uk/id/4000000074...,Isle of Wight Hill,NaN,NaN,NaN,landform,Hill Or Mountain,424844,137337,...,Hampshire,http://data.ordnancesurvey.co.uk/id/7000000000...,http://data.ordnancesurvey.co.uk/ontology/admi...,South East,http://data.ordnancesurvey.co.uk/id/7000000000...,England,http://data.ordnancesurvey.co.uk/id/country/en...,NaN,NaN,NaN
1,SU86.csv:osgb4000000074808919,http://data.ordnancesurvey.co.uk/id/4000000074...,Isle of Wight Pond,NaN,NaN,NaN,hydrography,Inland Water,495262,172358,...,Windsor and Maidenhead,http://data.ordnancesurvey.co.uk/id/7000000000...,http://data.ordnancesurvey.co.uk/ontology/admi...,South East,http://data.ordnancesurvey.co.uk/id/7000000000...,England,http://data.ordnancesurvey.co.uk/id/country/en...,NaN,NaN,NaN
2,SZ06.csv:osgb4000000026226540,http://data.ordnancesurvey.co.uk/id/4000000026...,Isle of Wight Road,NaN,NaN,NaN,transportNetwork,Named Road,403311,77620,...,Dorset,http://data.ordnancesurvey.co.uk/id/7000000000...,http://data.ordnancesurvey.co.uk/ontology/admi...,South West,http://data.ordnancesurvey.co.uk/id/7000000000...,England,http://data.ordnancesurvey.co.uk/id/country/en...,NaN,NaN,NaN
3,SZ28.csv:osgb4000000074551734,http://data.ordnancesurvey.co.uk/id/4000000074...,Bouldnor,NaN,NaN,NaN,populatedPlace,Village,436404,89600,...,Isle of Wight,http://data.ordnancesurvey.co.uk/id/7000000000...,http://data.ordnancesurvey.co.uk/ontology/admi...,South East,http://data.ordnancesurvey.co.uk/id/7000000000...,England,http://data.ordnancesurvey.co.uk/id/country/en...,NaN,http://dbpedia.org/resource/Bouldnor,NaN
4,SZ28.csv:osgb4000000074551705,http://data.ordnancesurvey.co.uk/id/4000000074...,Freshwater,NaN,NaN,NaN,populatedPlace,Village,433576,87130,...,Isle of Wight,http://data.ordnancesurvey.co.uk/id/7000000000...,http://data.ordnancesurvey.co.uk/ontology/admi...,South East,http://data.ordnancesurvey.co.uk/id/7000000000...,England,http://data.ordnancesurvey.co.uk/id/country/en...,NaN,"http://dbpedia.org/resource/Freshwater,_Isle_o...",http://sws.geonames.org/2649069


In [4]:
# Use: COUNTY_UNITARY_URI
# http://data.ordnancesurvey.co.uk/id/7000000000025469

In [8]:
df["TYPE"].unique(), df["LOCAL_TYPE"].unique()

(array(['landform', 'hydrography', 'transportNetwork', 'populatedPlace',
        'other', 'landcover'], dtype=object),
 array(['Hill Or Mountain', 'Inland Water', 'Named Road', 'Village',
        'Suburban Area', 'Hamlet', 'Town', 'Postcode',
        'Section Of Named Road', 'Numbered Road', 'Woodland Or Forest',
        'Other Landcover', 'Valley', 'Coastal Headland',
        'Other Coastal Landform', 'Wetland', 'Island', 'Cliff Or Slope',
        'Bay', 'Harbour', 'Medical Care Accommodation',
        'Electricity Production', 'Primary Education',
        'Vehicular Ferry Terminal', 'Section Of Numbered Road',
        'Spot Height', 'Tidal Water', 'Secondary Education',
        'Special Needs Education', 'Other Settlement', 'Other Landform',
        'Channel', 'Beach', 'Estuary', 'Sea', 'Railway',
        'Passenger Ferry Terminal',
        'Non State Primary Education,Non State Secondary Education',
        'Railway Station', 'Bus Station', 'Airport', 'Further Education',
        'H